In [123]:
import pandas as pd
import numpy as np
import matplotlib as plt
import seaborn as sns
import sklearn.metrics as metrics 
!pip install xgboost
import sklearn.metrics as metrics
from sklearn.datasets import make_classification
from xgboost import XGBClassifier
!pip install xgboost scikit-learn pandas numpy
import xgboost as xgb
import warnings
warnings.filterwarnings("ignore")
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier
from sklearn.svm import SVC
from sklearn.ensemble import ExtraTreesClassifier

In [124]:
def regressionMetrics(y, yhat):
    res = {'MSE': metrics.mean_squared_error(y,yhat),
           'RMSE': np.sqrt(metrics.mean_squared_error(y,yhat)),
           'MAE': metrics.mean_absolute_error(y,yhat),
           'MSLE': metrics.mean_squared_log_error(y,yhat),
           'RMSLE': np.sqrt(metrics.mean_squared_log_error(y,yhat))
          }
    return res


# DATA

In [125]:
df = pd.read_pickle('df.pkl')

In [126]:
df = df[df.notna().all(axis=1)]

In [127]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8693 entries, 4277 to 12969
Data columns (total 17 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   8693 non-null   int64  
 1   HomePlanet    8693 non-null   object 
 2   CryoSleep     8693 non-null   float64
 3   Cabin         8693 non-null   object 
 4   Destination   8693 non-null   object 
 5   Age           8693 non-null   float64
 6   VIP           8693 non-null   float64
 7   RoomService   8693 non-null   float64
 8   FoodCourt     8693 non-null   float64
 9   ShoppingMall  8693 non-null   float64
 10  Spa           8693 non-null   float64
 11  VRDeck        8693 non-null   float64
 12  Name          8693 non-null   object 
 13  a             8693 non-null   object 
 14  b             8693 non-null   object 
 15  c             8693 non-null   object 
 16  Transported   8693 non-null   object 
dtypes: float64(8), int64(1), object(8)
memory usage: 1.2+ MB


In [128]:
str_columns = ['HomePlanet','Destination','Name','a','b','c','Cabin']
df[str_columns] = df[str_columns].astype('string')

In [129]:
df['Transported'] = df['Transported'].astype(float)

In [121]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8693 entries, 4277 to 12969
Data columns (total 18 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   8693 non-null   int64  
 1   HomePlanet    8693 non-null   string 
 2   CryoSleep     8693 non-null   string 
 3   Cabin         8693 non-null   string 
 4   Destination   8693 non-null   string 
 5   Age           8693 non-null   int32  
 6   VIP           8693 non-null   int32  
 7   RoomService   8693 non-null   int32  
 8   FoodCourt     8693 non-null   int32  
 9   ShoppingMall  8693 non-null   int32  
 10  Spa           8693 non-null   int32  
 11  VRDeck        8693 non-null   float64
 12  Name          8693 non-null   string 
 13  a             8693 non-null   string 
 14  b             8693 non-null   string 
 15  c             8693 non-null   string 
 16  Transported   8693 non-null   object 
 17  Cbin          8693 non-null   string 
dtypes: float64(1), int32(6),

In [132]:
df['CryoSleep'] = df['CryoSleep'].astype('int')
df['Age'] = df['Age'].astype('int')
df['VIP'] = df['VIP'].astype('int')
df['RoomService'] = df['RoomService'].astype('int')
df['FoodCourt'] = df['FoodCourt'].astype('int')
df['ShoppingMall'] = df['ShoppingMall'].astype('int')
df['Spa'] = df['Spa'].astype('int')
#df[object_columns] = df[object_columns].astype('string')

In [133]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8693 entries, 4277 to 12969
Data columns (total 18 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   8693 non-null   int64  
 1   HomePlanet    8693 non-null   string 
 2   CryoSleep     8693 non-null   int32  
 3   Cabin         8693 non-null   string 
 4   Destination   8693 non-null   string 
 5   Age           8693 non-null   int32  
 6   VIP           8693 non-null   int32  
 7   RoomService   8693 non-null   int32  
 8   FoodCourt     8693 non-null   int32  
 9   ShoppingMall  8693 non-null   int32  
 10  Spa           8693 non-null   int32  
 11  VRDeck        8693 non-null   float64
 12  Name          8693 non-null   string 
 13  a             8693 non-null   string 
 14  b             8693 non-null   string 
 15  c             8693 non-null   string 
 16  Transported   8693 non-null   float64
 17  Cbin          8693 non-null   string 
dtypes: float64(2), int32(7),

In [134]:
df.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,a,b,c,Transported,Cbin
4277,101,Europa,0,B0P,TRAPPIST,39,0,0,0,0,0,0.0,Maham Ofracculy,B,0.0,P,0.0,0.0
4278,201,Earth,0,F0S,TRAPPIST,24,0,109,9,25,549,44.0,Juanna Vines,F,0.0,S,1.0,0.0
4279,301,Europa,0,A0S,TRAPPIST,58,1,43,3576,0,6715,49.0,Altark Susent,A,0.0,S,0.0,0.0
4280,302,Europa,0,A0S,TRAPPIST,33,0,0,1283,371,3329,193.0,Solam Susent,A,0.0,S,0.0,0.0
4281,401,Earth,0,F1S,TRAPPIST,16,0,303,70,151,565,2.0,Willy Santantines,F,1.0,S,1.0,0.0


# Catagroial Encoding

In [135]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

In [136]:
df_encoded= df.copy()

In [137]:
label_encoder = LabelEncoder()

In [138]:
for column in df:
    df_encoded[column] = label_encoder.fit_transform(df[column])

In [139]:
df_encoded

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,a,b,c,Transported,Cbin
4277,0,1,0,149,2,39,0,0,0,0,0,0,5252,1,0,0,0,0
4278,1,0,0,2184,2,24,0,109,9,25,458,47,4502,5,0,1,1,0
4279,2,1,0,1,2,58,1,43,1222,0,1285,52,457,0,0,1,0,0
4280,3,1,0,1,2,33,0,0,816,335,1143,191,7149,0,0,1,0,0
4281,4,0,0,3423,2,16,0,277,69,149,472,3,8319,5,1,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12965,8688,1,0,144,0,41,1,0,1406,0,940,78,3524,0,1795,0,0,0
12966,8689,0,1,5275,1,18,0,0,0,0,0,0,4780,6,548,1,0,1
12967,8690,0,0,5280,2,26,0,0,0,980,1,0,3002,6,551,1,1,0
12968,8691,1,0,2128,0,32,0,0,746,0,322,1135,1596,4,1385,1,0,0


In [140]:
for column in df_encoded:
    df[column] = df_encoded[column]

In [141]:
df.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,a,b,c,Transported,Cbin
4277,0,1,0,149,2,39,0,0,0,0,0,0,5252,1,0,0,0,0
4278,1,0,0,2184,2,24,0,109,9,25,458,47,4502,5,0,1,1,0
4279,2,1,0,1,2,58,1,43,1222,0,1285,52,457,0,0,1,0,0
4280,3,1,0,1,2,33,0,0,816,335,1143,191,7149,0,0,1,0,0
4281,4,0,0,3423,2,16,0,277,69,149,472,3,8319,5,1,1,1,0


In [142]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8693 entries, 4277 to 12969
Data columns (total 18 columns):
 #   Column        Non-Null Count  Dtype
---  ------        --------------  -----
 0   PassengerId   8693 non-null   int64
 1   HomePlanet    8693 non-null   int32
 2   CryoSleep     8693 non-null   int64
 3   Cabin         8693 non-null   int32
 4   Destination   8693 non-null   int32
 5   Age           8693 non-null   int64
 6   VIP           8693 non-null   int64
 7   RoomService   8693 non-null   int64
 8   FoodCourt     8693 non-null   int64
 9   ShoppingMall  8693 non-null   int64
 10  Spa           8693 non-null   int64
 11  VRDeck        8693 non-null   int64
 12  Name          8693 non-null   int32
 13  a             8693 non-null   int32
 14  b             8693 non-null   int32
 15  c             8693 non-null   int32
 16  Transported   8693 non-null   int64
 17  Cbin          8693 non-null   int32
dtypes: int32(8), int64(10)
memory usage: 1018.7 KB


In [143]:
df.isna().sum()

PassengerId     0
HomePlanet      0
CryoSleep       0
Cabin           0
Destination     0
Age             0
VIP             0
RoomService     0
FoodCourt       0
ShoppingMall    0
Spa             0
VRDeck          0
Name            0
a               0
b               0
c               0
Transported     0
Cbin            0
dtype: int64

In [144]:
df.to_pickle('updated_dataframe.pkl')

# Supervised Models

In [145]:
from sklearn.model_selection import train_test_split

In [146]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8693 entries, 4277 to 12969
Data columns (total 18 columns):
 #   Column        Non-Null Count  Dtype
---  ------        --------------  -----
 0   PassengerId   8693 non-null   int64
 1   HomePlanet    8693 non-null   int32
 2   CryoSleep     8693 non-null   int64
 3   Cabin         8693 non-null   int32
 4   Destination   8693 non-null   int32
 5   Age           8693 non-null   int64
 6   VIP           8693 non-null   int64
 7   RoomService   8693 non-null   int64
 8   FoodCourt     8693 non-null   int64
 9   ShoppingMall  8693 non-null   int64
 10  Spa           8693 non-null   int64
 11  VRDeck        8693 non-null   int64
 12  Name          8693 non-null   int32
 13  a             8693 non-null   int32
 14  b             8693 non-null   int32
 15  c             8693 non-null   int32
 16  Transported   8693 non-null   int64
 17  Cbin          8693 non-null   int32
dtypes: int32(8), int64(10)
memory usage: 1018.7 KB


In [147]:
X = df.loc[:,df.columns != 'Transported']
y = df['Transported']

In [148]:
X_train,X_test, y_train,y_test = train_test_split(X,y,test_size = 0.3, random_state = 47)

In [149]:
def classificationMetrics(y, yhat):
    prf1 = metrics.precision_recall_fscore_support(y,yhat)
    res = {'Accuracy': metrics.accuracy_score(y,yhat),
           'Precision':prf1[0][1],
           'Recall': prf1[1][1],
           'f1-score': prf1[2][1],
           'Log-loss': metrics.log_loss(y,yhat),
           'AUC': metrics.roc_auc_score(y,yhat)
          }
    return res

In [150]:
models_list = pd.DataFrame()

# Logostic Regression

In [151]:
mod1 = LogisticRegression()
mod1.fit(X,y)

pred1 = mod1.predict(X)

In [152]:
classificationMetrics(y, pred1)

{'Accuracy': 0.7740710916829633,
 'Precision': 0.7431506849315068,
 'Recall': 0.8426222019186843,
 'f1-score': 0.7897666452579747,
 'Log-loss': 7.803425322251015,
 'AUC': 0.7735706606348925}

In [153]:
model_dict = {'model': "Logistic Regression"}
models_list = models_list.append({**model_dict, **classificationMetrics(y, pred1)}, ignore_index=True)
models_list

,model,Accuracy,Precision,Recall,f1-score,Log-loss,AUC
0,Logistic Regression,0.774071,0.743151,0.842622,0.789767,7.803425,0.773571


In [154]:
pd.crosstab(y, pred1)

col_0,0,1
Transported,,
0,3040,1275
1,689,3689


# Decision Tree

In [155]:
mod2 = DecisionTreeClassifier(random_state=1)
mod2.fit(X,y)

DecisionTreeClassifier(random_state=1)

In [156]:
pred2 = mod2.predict(X)
model_dict = {'model': "Decision Tree"}
models_list = models_list.append({**model_dict, **classificationMetrics(y, pred2)}, ignore_index=True)
models_list

,model,Accuracy,Precision,Recall,f1-score,Log-loss,AUC
0,Logistic Regression,0.774071,0.743151,0.842622,0.789767,7.803425e+00,0.773571
1,Decision Tree,1.000000,1.000000,1.000000,1.000000,9.992007e-16,1.000000


In [157]:
pd.crosstab(y, pred2)

col_0,0,1
Transported,,
0,4315,0
1,0,4378


# Random Forest

In [158]:
mod3 = RandomForestClassifier(random_state=1)
mod3.fit(X,y)

RandomForestClassifier(random_state=1)

In [159]:
pred3 = mod3.predict(X)
model_dict = {'model': "RandomForest"}
models_list = models_list.append({**model_dict, **classificationMetrics(y, pred3)}, ignore_index=True)
models_list

,model,Accuracy,Precision,Recall,f1-score,Log-loss,AUC
0,Logistic Regression,0.774071,0.743151,0.842622,0.789767,7.803425e+00,0.773571
1,Decision Tree,1.000000,1.000000,1.000000,1.000000,9.992007e-16,1.000000
2,RandomForest,1.000000,1.000000,1.000000,1.000000,9.992007e-16,1.000000


In [160]:
pd.crosstab(y, pred3)

col_0,0,1
Transported,,
0,4315,0
1,0,4378



# Adaptive Boosting (ADABoost)

In [161]:
mod4 = AdaBoostClassifier(random_state=1)
mod4.fit(X,y)

AdaBoostClassifier(random_state=1)

In [162]:
pred4 = mod4.predict(X)
model_dict = {'model': "ADABoost"}
models_list = models_list.append({**model_dict, **classificationMetrics(y, pred4)}, ignore_index=True)
models_list

,model,Accuracy,Precision,Recall,f1-score,Log-loss,AUC
0,Logistic Regression,0.774071,0.743151,0.842622,0.789767,7.803425e+00,0.773571
1,Decision Tree,1.000000,1.000000,1.000000,1.000000,9.992007e-16,1.000000
2,RandomForest,1.000000,1.000000,1.000000,1.000000,9.992007e-16,1.000000
3,ADABoost,0.799264,0.785018,0.828232,0.806046,6.933275e+00,0.799052


In [163]:
pd.crosstab(y, pred4)

col_0,0,1
Transported,,
0,3322,993
1,752,3626


# Gradient Boosting Machine (GBM)

In [165]:
mod5 = GradientBoostingClassifier(random_state=1)
mod5.fit(X,y)

GradientBoostingClassifier(random_state=1)

In [166]:
pred5 = mod5.predict(X)
model_dict = {'model': "GBM"}
models_list = models_list.append({**model_dict, **classificationMetrics(y, pred5)}, ignore_index=True)
models_list

,model,Accuracy,Precision,Recall,f1-score,Log-loss,AUC
0,Logistic Regression,0.774071,0.743151,0.842622,0.789767,7.803425e+00,0.773571
1,Decision Tree,1.000000,1.000000,1.000000,1.000000,9.992007e-16,1.000000
2,RandomForest,1.000000,1.000000,1.000000,1.000000,9.992007e-16,1.000000
3,ADABoost,0.799264,0.785018,0.828232,0.806046,6.933275e+00,0.799052
4,GBM,0.823076,0.809908,0.847647,0.828348,6.110817e+00,0.822897


In [167]:
pd.crosstab(y, pred5)

col_0,0,1
Transported,,
0,3444,871
1,667,3711


# Support Vector Machine (SVM)

In [169]:
mod6 = SVC(probability=True)
mod6.fit(X,y)

SVC(probability=True)

In [170]:
pred6 = mod6.predict(X)
model_dict = {'model': "SVM"}
models_list = models_list.append({**model_dict, **classificationMetrics(y, pred6)}, ignore_index=True)
models_list

,model,Accuracy,Precision,Recall,f1-score,Log-loss,AUC
0,Logistic Regression,0.774071,0.743151,0.842622,0.789767,7.803425e+00,0.773571
1,Decision Tree,1.000000,1.000000,1.000000,1.000000,9.992007e-16,1.000000
2,RandomForest,1.000000,1.000000,1.000000,1.000000,9.992007e-16,1.000000
3,ADABoost,0.799264,0.785018,0.828232,0.806046,6.933275e+00,0.799052
4,GBM,0.823076,0.809908,0.847647,0.828348,6.110817e+00,0.822897
5,SVM,0.775221,0.729632,0.879625,0.797639,7.763708e+00,0.774459


In [171]:
pd.crosstab(y, pred6)

col_0,0,1
Transported,,
0,2888,1427
1,527,3851


# XGBoost

In [172]:
import xgboost as xgb

In [173]:
mod7 = xgb.XGBClassifier()
mod7.fit(X, y)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, ...)

In [174]:
pred7 = mod7.predict(X)
model_dict = {'model': "XGB"}
models_list = models_list.append({**model_dict, **classificationMetrics(y, pred7)}, ignore_index=True)
models_list

,model,Accuracy,Precision,Recall,f1-score,Log-loss,AUC
0,Logistic Regression,0.774071,0.743151,0.842622,0.789767,7.803425e+00,0.773571
1,Decision Tree,1.000000,1.000000,1.000000,1.000000,9.992007e-16,1.000000
2,RandomForest,1.000000,1.000000,1.000000,1.000000,9.992007e-16,1.000000
3,ADABoost,0.799264,0.785018,0.828232,0.806046,6.933275e+00,0.799052
4,GBM,0.823076,0.809908,0.847647,0.828348,6.110817e+00,0.822897
5,SVM,0.775221,0.729632,0.879625,0.797639,7.763708e+00,0.774459
6,XGB,0.943748,0.947526,0.940384,0.943941,1.942902e+00,0.943772


In [183]:
sns.scatterplot(y, pred7)

<AxesSubplot:xlabel='Cbin', ylabel='Density'>

# Model Selection

In [175]:
models_list.sort_values('Accuracy',ascending=False)

,model,Accuracy,Precision,Recall,f1-score,Log-loss,AUC
1,Decision Tree,1.000000,1.000000,1.000000,1.000000,9.992007e-16,1.000000
2,RandomForest,1.000000,1.000000,1.000000,1.000000,9.992007e-16,1.000000
6,XGB,0.943748,0.947526,0.940384,0.943941,1.942902e+00,0.943772
4,GBM,0.823076,0.809908,0.847647,0.828348,6.110817e+00,0.822897
3,ADABoost,0.799264,0.785018,0.828232,0.806046,6.933275e+00,0.799052
5,SVM,0.775221,0.729632,0.879625,0.797639,7.763708e+00,0.774459
0,Logistic Regression,0.774071,0.743151,0.842622,0.789767,7.803425e+00,0.773571


In [176]:
models_list.sort_values('AUC',ascending=False)

,model,Accuracy,Precision,Recall,f1-score,Log-loss,AUC
1,Decision Tree,1.000000,1.000000,1.000000,1.000000,9.992007e-16,1.000000
2,RandomForest,1.000000,1.000000,1.000000,1.000000,9.992007e-16,1.000000
6,XGB,0.943748,0.947526,0.940384,0.943941,1.942902e+00,0.943772
4,GBM,0.823076,0.809908,0.847647,0.828348,6.110817e+00,0.822897
3,ADABoost,0.799264,0.785018,0.828232,0.806046,6.933275e+00,0.799052
5,SVM,0.775221,0.729632,0.879625,0.797639,7.763708e+00,0.774459
0,Logistic Regression,0.774071,0.743151,0.842622,0.789767,7.803425e+00,0.773571


In [179]:
sns.pairplot(df, hue='Transported')

In [182]:
df.to_pickle("df.pk2")